In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

# Configuración para optimizar visualización
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', None)


print("✅ Librerías importadas exitosamente")
print("✅ Configuración de visualización establecida")

✅ Librerías importadas exitosamente
✅ Configuración de visualización establecida


In [2]:
# ===============================================================
# CARGA DEL DATASET INICIAL
# ===============================================================

# Ruta al archivo CSV con datos de propiedades colombianas
csv_path = "data/properties_gold.csv"

# Cargar dataset completo
df = pd.read_csv(csv_path)

# Información básica del dataset
print("🏠 DATASET CARGADO: Propiedades Inmobiliarias Colombia")
print("=" * 55)
print(f"📊 Dimensiones: {df.shape}")
print(f"📋 Registros: {df.shape[0]:,}")
print(f"📋 Variables: {df.shape[1]}")
print(f"💾 Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Vista previa de primeras filas
print(f"\n📋 PRIMERAS 3 FILAS:")
df.head(3)

🏠 DATASET CARGADO: Propiedades Inmobiliarias Colombia
📊 Dimensiones: (140435, 16)
📋 Registros: 140,435
📋 Variables: 16
💾 Tamaño en memoria: 161.0 MB

📋 PRIMERAS 3 FILAS:


,ad_type,start_date,end_date,created_on,lat,lon,price,title,description,property_type,operation_type,surface_total_final,bedrooms_final,bathrooms_final,l3_final,l4_final
0,Propiedad,2020-10-07,2020-10-08,2020-10-07,NaN,NaN,4.000000e+09,Casa en Venta Ubicado en MEDELLIN,"Codigo Inmueble 6348 La conocerás y no te querrás ir Cómoda casa con 7 habitaciones, 8 baños com...",Casa,Venta,NaN,7.0,8.0,Medellín,NaN
1,Propiedad,2020-10-07,9999-12-31,2020-10-07,6.151412,-75.379867,3.800000e+08,Venta de casa en Santa Clara 1 Rionegro,"Rionegro\nUbicación Santa Clara 1\nLote 76, 5 metros\nÁrea construcción 172 metros\nBaños 3\nPar...",Casa,Venta,NaN,4.0,3.0,Rionegro,NaN
2,Propiedad,2020-10-07,2021-01-19,2020-10-07,6.129877,-75.377835,3.900000e+08,VENTA CASA INDEPENDIENTE EN RIONEGRO ANTIOQUIA,"4 alcobas, 3 baños, estudio, cocina integral, sala comedor, patio , garaje cubierto \n\nUbicada...",Casa,Venta,NaN,4.0,3.0,Rionegro,NaN


## Revisemos el rango de años de los datos

In [3]:
# Replace 'date_column' with your actual column name
df['created_on'] = pd.to_datetime(df['created_on'], errors='coerce')
years = df['created_on'].dt.year

min_year = years.min()
max_year = years.max()

print(f"Year range: {min_year} - {max_year}")


Year range: 2020 - 2021


In [4]:
from sqlalchemy import create_engine

# Connection details
user = "andres"
password = "Pass123"
host = "127.0.0.1"
port = "5433"
database = "testdb"

# Create SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Test the connection by running a simple query
test_con = pd.read_sql("SELECT version();", engine)
print(test_con)

                                                                                               version
0  PostgreSQL 14.19 (Ubuntu 14.19-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu...


In [5]:
# Save DataFrame to SQL table named 'my_table'
df.to_sql('properties', engine, if_exists='replace', index=False)

435

In [6]:
# Query all rows from the 'properties' table
result = pd.read_sql("SELECT * FROM properties WHERE surface_total_final is not null and " \
"l3_final = 'Medellín'", engine)

# Show the first rows
print(result.head())

     ad_type  start_date    end_date created_on       lat        lon        price                                                                                     title                                                                                          description property_type operation_type  surface_total_final  bedrooms_final  bathrooms_final  l3_final         l4_final
0  Propiedad  2020-10-07  2021-05-01 2020-10-07  6.297802 -75.577835  310000000.0                                                               Venta de casa en Florencia   Venta de Casa en Florencia Medellin\n\nLa casa tiene 5 habitaciones\n2 Baños uno con bañera \nBi...          Casa          Venta                192.0             5.0              2.0  Medellín  Doce de Octubre
1  Propiedad  2020-10-07  2021-01-05 2020-10-07  5.871480 -75.561657  190000000.0                                     Vendo Casa en Santa Bárbara sector Alto de las Guacas  Vendo Casa para habitación, Descanso o Construcción, área de 

In [7]:
# Query all rows from the 'properties' table
result = pd.read_sql("SELECT DISTINCT (l3_final) as barrio from properties ORDER BY barrio", engine)


print(result)

                       barrio
0                   Abejorral
1                  Alejandría
2                      Amalfi
3                       Andes
4                    Apartadó
5                     Barbosa
6                       Bello
7                     Betania
8                      Caldas
9                      Carepa
10                   Caucasia
11                  Chigorodó
12             Ciudad Bolívar
13                    Cocorná
14                 Concepción
15                  Concordia
16                 Copacabana
17                    Ebéjico
18       El Carmen de Viboral
19                   Envigado
20                   Fredonia
21                    Giraldo
22                  Girardota
23                     Guarne
24                    Guatapé
25                   Hispania
26                     Itagui
27                     Jardín
28                     Jericó
29                    La Ceja
30                La Estrella
31                 La Pintada
32        

In [8]:
query = """
SELECT l4_final, COUNT(*) as count
FROM properties
WHERE created_on IS NOT NULL
  AND price IS NOT NULL
  AND surface_total_final IS NOT NULL
  AND bedrooms_final IS NOT NULL
  AND bathrooms_final IS NOT NULL
  AND l3_final IS NOT NULL
  AND l4_final IS NOT NULL
GROUP BY l4_final
ORDER BY count DESC
"""
result = pd.read_sql(query, engine)
print(result)

                l4_final  count
0             El Poblado   4082
1               Laureles   3072
2                  Belén   2807
3             La América   1398
4                Robledo   1242
5                Cabañas    870
6             Candelaria    720
7            La Estación    638
8           Buenos Aires    561
9                 Niquía    451
10              Castilla    450
11               Mayorca    353
12           La Frontera    312
13         San Cristóbal    308
14              Guayabal    285
15             Altavista    266
16            San Javier    242
17              Ditaires    228
18              Aranjuez    221
19            Las Palmas    199
20            La Doctora    189
21              Jardines    187
22    Loma de Los Bernal    179
23       Doce de Octubre    174
24           Santa María    171
25         Villa Hermosa    158
26           Los Colores    144
27           Santa Elena    137
28  San Antonio de Prado    127
29              Manrique    123
30      

In [9]:
query = """
SELECT COUNT(*) as count
FROM properties
WHERE created_on IS NOT NULL
  AND price IS NOT NULL
  AND surface_total_final IS NOT NULL
  AND bedrooms_final IS NOT NULL
  AND bathrooms_final IS NOT NULL
  AND l3_final IS NOT NULL
  AND l4_final IS NOT NULL

"""
result = pd.read_sql(query, engine)
print(result)

   count
0  21192


In [10]:
query = """
SELECT * FROM properties WHERE surface_total_final is not null
and l4_final = 'El Poblado'
"""

result = pd.read_sql(query, engine)
print(result.head(5))

     ad_type  start_date    end_date created_on       lat        lon        price                                                title                                                                                          description property_type operation_type  surface_total_final  bedrooms_final  bathrooms_final  l3_final    l4_final
0  Propiedad  2020-10-07  2020-10-09 2020-10-07  6.204000 -75.576000  500000000.0  Casa En Venta En Medellin El Poblado Cod. VRAS-3064  Apartamento en venta en el poblado cerca de la Cnica las Vegas, en un sector muy residencial y t...          Casa          Venta                147.0             3.0              3.0  Medellín  El Poblado
1  Propiedad  2020-10-07  9999-12-31 2020-10-07  6.151414 -75.611236  280000000.0                VENTA APARTAMENTO SABANETA EL CARMELO  Venta de espectacular Apartamento en Sabaneta el Carmelo, 65 mts2, cocina integral, sala comedor...   Apartamento          Venta                 65.0             3.0              2.0

In [11]:
query = """
SELECT * FROM properties WHERE price = 4.650000e+11

"""

result = pd.read_sql(query, engine)
print(result.head(5))



     ad_type  start_date    end_date created_on       lat        lon         price                                  title                                                                                          description property_type operation_type  surface_total_final  bedrooms_final  bathrooms_final  l3_final    l4_final
0  Propiedad  2021-02-24  2021-04-15 2021-02-24  6.243123 -75.611478  4.650000e+11  Venta de casa Simón Bolívar ,Medellin  Se vende casa en Simón Bolivar , espacios amplios y ventilados ,excelente ubicacion y amplias ru...          Casa          Venta                168.0             4.0              2.0  Medellín  La América


In [12]:
SMLV_2020 = 877803
SMLV_2021 = 908526
SMLV_2025 = 1423500

ratio_20 = (SMLV_2025/SMLV_2020)
ratio_21 = (SMLV_2025/SMLV_2021)
print("ratio_20", ratio_20)
print("ratio_21", ratio_21)

print(500000000 * ratio_20)


ratio_20 1.6216622636286273
ratio_21 1.566823624200078
810831131.8143137


In [13]:
df[['created_on', 'lat', 'lon','price','surface_total_final', 'bedrooms_final', 'bathrooms_final', 'l3_final', 'l4_final']].head(2)

,created_on,lat,lon,price,surface_total_final,bedrooms_final,bathrooms_final,l3_final,l4_final
0,2020-10-07,NaN,NaN,4.000000e+09,NaN,7.0,8.0,Medellín,NaN
1,2020-10-07,6.151412,-75.379867,3.800000e+08,NaN,4.0,3.0,Rionegro,NaN
